In [1]:
import scipy.stats as sts
from pydantic import BaseModel, ValidationError
from typing import Optional
import numpy as np
import numpy.random as rd
from abc import ABCMeta, abstractmethod


In [2]:
import pydantic.types as ptp

In [6]:
class A(BaseModel):
    x: ptp.PositiveFloat

In [11]:
A(x=1)

A(x=1.0)

In [56]:
class Workshop:
    def __init__(self):
        self.Resources = dict()
        self.Creators = dict()

    def append_resource(self, name, res):
        self.Resources[name] = res
        
    def register(self, cls_name, cls):
        self.Creators[cls_name] = cls
    
    def parse(self, src, loc=None):
        return eval(src, self.Creators, loc)

In [57]:
ws = Workshop()


In [59]:
ws.parse('Exp(0.5)')

TypeError: __init__() takes 1 positional argument but 2 were given

In [119]:
sig.parameters

mappingproxy({'scale': <Parameter "scale: float = 1">})

In [308]:
class Creator:
    def __init__(self, cls_name, cls):
        self.Name = cls_name
        self.Class = cls
        self.Args = inspect.signature(cls).parameters
        
    def complete_def(self, seq):
        root = ast.parse(seq)
        for inputs in ast.walk(root):
            if isinstance(inputs, ast.Call):
                break

        args = [arg.value if isinstance(arg, ast.Constant) else ast.unparse(arg) for arg in inputs.args]

        named = [arg.arg for arg in inputs.keywords]
        unnamed = {k: v for k, v in self.Args.items() if k not in named}
        
        if len(unnamed) == 0:
            return seq
        
        inputs.args = list()
        
        full = ast.unparse(inputs)
        
        temp = list()
        for i, (k, v) in enumerate(unnamed.items()):
            try:
                v = args[i]
            except IndexError:
                v = v.default
            temp.append(f'{k}={v}')

        unnamed = f'({", ".join(temp)}'

        full = full.replace('(', unnamed + (', ' if len(named) > 0 else ''), 1)
        return full 
    
    def parse(self, seq, loc=None):
        seq = self.complete_def(seq)
        return eval(seq, {self.Name: self.Class}, loc)
    
    def parse_dict(self, args):
        return self.Class.parse_dict(args)

    
class Exp(BaseModel):
    scale: float = 1
    s1: float = 2
    loc = 0
    bb = 50
    
crt = Creator('Exp', Exp)
crt.complete_def("Exp(3)")
crt.parse('Exp(3)')

Exp(scale=3.0, s1=2.0, loc=0, bb=50)

In [306]:
%timeit crt.complete_def("Exp(3)")
%timeit crt.complete_def("Exp(scale=3)")

20.9 µs ± 258 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
26.1 µs ± 181 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [311]:
%timeit crt.parse('Exp(3)')
%timeit crt.parse('Exp(scale=3)')
%timeit crt.parse('Exp(scale=3.0, s1=2.0, loc=0, bb=50)')
%timeit crt.parse('Exp(scale=3.0, s1=2.0, loc=0, bb=bb)', {'bb': 50})

47.5 µs ± 956 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
52.3 µs ± 193 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
42 µs ± 197 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
42.6 µs ± 261 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [316]:
%timeit crt.parse('Exp(scale=3.0, s1=2.0, loc=0, bb=bb * 4)', {'bb': 50})

44.9 µs ± 434 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [318]:
qq = crt.parse('Exp(scale=3.0, s1=2.0, loc=0, bb=bb * np.sqrt( 4))', {'bb': 50, 'np': np})
qq

Exp(scale=3.0, s1=2.0, loc=0, bb=100)

In [359]:
inputs.func.id

'Exp'

In [323]:
seq.split('(')[0]

'Exp'

In [206]:
ast.keyword(arg='scale', value = args[0])

In [251]:
root

In [326]:
class A(BaseModel):
    x = 1
    y = 2
    
a = A()
a.dict()

{'x': 1, 'y': 2}

In [343]:
class B(A):
    z = 4

    __kk__ = 0
    def __init__(self, **data):
        super().__init__(**data)

        
    
b = B()
b.dict()


{'x': 1, 'y': 2, 'z': 4}

In [347]:
type(b).__name__

'B'